## Example 1 Taxi trips within and outside Manhattan

Below we apply Naive Bayes Classifier to classying taxi trips. Given a certain sample of workday daytime taxi trips with the information of how long and fast the trip was as well and how many passengers were on it as how big was the tip (all but the passenger count encoded as discrete categorical variables starting from 1 (smallest) and up to 4-6 (highest)) we need to train a classifier of whether the trip happened inside Manhattan (2) or not (1). Trips from  the sample are assigned such labels to serve for the supervised learning purposes as well as for testing the classifier.

In [9]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import scipy
from scipy import stats
%matplotlib inline
from matplotlib import pyplot as plt
import random
#First load the sample and split it randomly into the training (75%) and test (25%) set. 
data = pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/ML_2017/NYC_taxi_sample.csv",index_col=0)
data.manhattan=data.manhattan-1
np.random.seed(2015)
ind=stats.bernoulli.rvs(p = 0.75, size = len(data.index))
train=data[ind==1]
test=data[ind==0]
train.head()

,manhattan,tip,dist,speed,pass
26066,0,1,1,1,1
26100,0,3,5,6,1
26589,1,2,2,2,1
27022,0,3,4,5,1
28621,0,3,1,1,1


In [10]:
#how big is the entire sample as well as the training/test sets
[len(data.index),len(train.index),len(test.index)]

[5329, 4004, 1325]

Now implement the estimator for the training sample statistics $P(x=x^*|y=b)$ and $P(y=b)$.

In [3]:
def trainNaiveBayesDiscrete(trainData):
  #training discrete Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
  #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
  dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
  #split the training data between two labels
  ind1=tY==0
  ind2=tY==1
  #estimate P(y=b)  
  dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
  dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
  #estimate conditional probabilities P(x|y=b)
  for j in trainData.columns[1:]:
    for i in range(1,m+1):
        dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
        dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
  return dp

In [4]:
def classifyNaiveBayesDiscrete(classData,dp):
  #classifying using trained discrete Naive Bayes Classifier
  Y=classData[classData.columns[0]]*0 #initialize the empty array 
  for i in classData.index: #for al records to classify
    #start with the priors
    P1=dp[0][classData.columns[0]][1]; 
    P2=dp[1][classData.columns[0]][1];
    #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
    for j in classData.columns[1:]:
      P1=P1*dp[0][j][classData[j][i]]
      P2=P2*dp[1][j][classData[j][i]]
    Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
  return Y

In [5]:
#now train the classifier based on the training sample
dp=trainNaiveBayesDiscrete(train)

In [6]:
dp

[  manhattan        tip      dist     speed       pass
 1  0.516484   0.560928  0.209381  0.126692   0.708414
 2       NaN  0.0913926  0.176499  0.220503    0.12911
 3       NaN  0.0976789  0.116054  0.202611  0.0531915
 4       NaN   0.162476  0.118472  0.194391   0.109284
 5       NaN   0.049323  0.147002  0.133946          0
 6       NaN  0.0382012  0.232592  0.121857          0,
   manhattan        tip       dist      speed       pass
 1  0.483516   0.366736    0.42407   0.305785   0.643079
 2       NaN   0.179236   0.271694   0.415806   0.201963
 3       NaN   0.167355   0.141012   0.157025   0.089876
 4       NaN   0.164773  0.0991736  0.0717975  0.0650826
 5       NaN  0.0542355  0.0247934   0.036157          0
 6       NaN  0.0676653  0.0392562  0.0134298          0]

In [7]:
#run the classifier over the test sample
C=classifyNaiveBayesDiscrete(test,dp)

In [8]:
#compute the classification accuracy over the test set comparing the classification obtained with the labels provided
1.0*sum(C==test.manhattan)/len(C)

0.7124528301886792

In [9]:
#also compute the accuracy over the training set
Ct=classifyNaiveBayesDiscrete(train,dp)
1.0*sum(Ct==train.manhattan)/len(Ct)

0.7047952047952047

# Naive Bayes Classifier (Gaussian)

In [10]:
def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  #estimate priors
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  #estimate sample distribution paramters for p(xi|y=b)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

In [11]:
def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    #start from the priors
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu2'];
    #multiply by conditional probability densities p(xi|y=b)
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

# Example 2. Predicting type of the house

Based on the sample of characteristics and the prices of the single unit residential and commercial houses sold in "our" zip code 11201 in downtown Brooklyn between the years 2009 and 2012, build a classifier defining if the sold house was actually residential or commercial.

In [12]:
#read the sample
data = pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/ML_2017/NYC_RE_sample.csv")
# split sample into training and test ones
data.columns=['index','area','land','year','price','bldtype'] #house size, land, year built, price and type
data.bldtype=data.bldtype-1 #bring the house classes to 0 - residential, 1 - commercial
data.head()

,index,area,land,year,price,bldtype
0,1,2607,1200,2010,825000.0,0
1,2,1950,1783,1899,1685000.0,0
2,3,2520,1875,1899,1100000.0,0
3,4,3750,3125,1931,1200000.0,1
4,5,7812,5021,1908,1900000.0,1


In [13]:
#transform the features to relative values getting rid of obvious correlations: consider price per sq. foot, land per sq. foot of indoor area
data1=pd.DataFrame({'Y':data.bldtype,'price':data.price/data.area,'land':data.land/data.area,'year':data.year})

In [14]:
#split dataset into 60% training and 40% test 
np.random.seed(2015)
ind=stats.bernoulli.rvs(p = 0.6, size = len(data.index))
train=data1[ind==1]
test=data1[ind==0]
train.head()

,Y,land,price,year
1,0,0.914359,864.102564,1899
3,1,0.833333,320.000000,1931
6,0,0.666966,1049.160671,1901
8,1,0.200000,340.000000,1900
10,0,0.510204,510.204082,1901


In [15]:
dp=trainNaiveBayes(train) #train the Gausian Naive Bayes Classifier

In [16]:
dp

,Y,land,price,year
mu1,0.679245,0.639192,1010.22,1907.14
sigma1,NaN,0.263976,388.859,46.0733
mu2,0.320755,0.914258,384.538,1928.53
sigma2,NaN,0.9183,264.816,21.812


In [17]:
C=classifyNaiveBayes(test,dp) #classify test set

In [18]:
#classification accuracy (over test set)
100.0*sum(C==test.Y)/len(C)

82.92682926829268

In [19]:
#also report accuracy of the classifier over the training - it is slightly higher, although not that higher
Ct=classifyNaiveBayes(train,dp)
100.0*sum(Ct==train.Y)/len(Ct)

88.67924528301887

## Use the Package from Sklearn

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB

http://scikit-learn.org/stable/modules/naive_bayes.html

In [20]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(train.iloc[:,1:],train.Y ).predict(test.iloc[:,1:])
(y_pred==test.Y).sum()*1.0/len(y_pred)

0.82926829268292679

# Practice: Spam classification

1. Title:  SPAM E-mail Database

2. Sources:
   (a) Creators: Mark Hopkins, Erik Reeber, George Forman, Jaap Suermondt
        Hewlett-Packard Labs, 1501 Page Mill Rd., Palo Alto, CA 94304
   (b) Donor: George Forman (gforman at nospam hpl.hp.com)  650-857-7835
   (c) Generated: June-July 1999

3. Past Usage:
   (a) Hewlett-Packard Internal-only Technical Report. External forthcoming.
   (b) Determine whether a given email is spam or not.
   (c) ~7% misclassification error.
       False positives (marking good mail as spam) are very undesirable.
       If we insist on zero false positives in the training/testing set,
       20-25% of the spam passed through the filter.

4. Relevant Information:
        The "spam" concept is diverse: advertisements for products/web
        sites, make money fast schemes, chain letters, pornography...
	The collection of spam e-mails came from the postmaster and 
	individuals who had filed spam.  The collection of non-spam 
	e-mails came from filed work and personal e-mails, and hence
	the word 'george' and the area code '650' are indicators of 
	non-spam.  These are useful when constructing a personalized 
	spam filter.  One would either have to blind such non-spam 
	indicators or get a very wide collection of non-spam to 
	generate a general purpose spam filter.

        For background on spam:
        Cranor, Lorrie F., LaMacchia, Brian A.  Spam! 
        Communications of the ACM, 41(8):74-83, 1998.

5. Number of Instances: 4601 (1813 Spam = 39.4%)

6. Number of Attributes: 58 (57 continuous, 1 nominal class label)

7. Attribute Information:
The last column of 'spambase.data' denotes whether the e-mail was 
considered spam (1) or not (0), i.e. unsolicited commercial e-mail.  
Most of the attributes indicate whether a particular word or
character was frequently occuring in the e-mail.  The run-length
attributes (55-57) measure the length of sequences of consecutive 
capital letters.  For the statistical measures of each attribute, 
see the end of this file.  Here are the definitions of the attributes:

48 continuous real [0,100] attributes of type word_freq_WORD 
= percentage of words in the e-mail that match WORD,
i.e. 100 * (number of times the WORD appears in the e-mail) / 
total number of words in e-mail.  A "word" in this case is any 
string of alphanumeric characters bounded by non-alphanumeric 
characters or end-of-string.

6 continuous real [0,100] attributes of type char_freq_CHAR
= percentage of characters in the e-mail that match CHAR,
i.e. 100 * (number of CHAR occurences) / total characters in e-mail

1 continuous real [1,...] attribute of type capital_run_length_average
= average length of uninterrupted sequences of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_longest
= length of longest uninterrupted sequence of capital letters

1 continuous integer [1,...] attribute of type capital_run_length_total
= sum of length of uninterrupted sequences of capital letters
= total number of capital letters in the e-mail

1 nominal {0,1} class attribute of type spam
= denotes whether the e-mail was considered spam (1) or not (0), 
i.e. unsolicited commercial e-mail.  


8. Missing Attribute Values: None

9. Class Distribution:
	Spam	  1813  (39.4%)
	Non-Spam  2788  (60.6%)



In [21]:
import urllib
data = urllib.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data").read()
data_name=urllib.urlopen("https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.names").read()
# Read the data
data=data.split("\r\n")
data_spam=[]
for i in range(len(data)):
    if len(data[i])>0:
        temp=data[i].split(",")
        #change from str to float
        t_l=[]
        for j in range(len(temp)):
            t_l.append(float(temp[j]))
        data_spam.append(t_l)

#Read the column names
temp=data_name.split("\r\n")
column_names=[]
for i in temp:
    if (i.startswith('word') or i.startswith('char') or i.startswith('capital')):
        column_names.append(i.split(":")[0])
column_names.append("spam") 

In [22]:
#Generate the data set for our tests
data_spam=pd.DataFrame(data_spam)
data_spam.columns=column_names
X=data_spam.iloc[:,0:-1]
y=data_spam.iloc[:,-1]
data_spam=pd.concat([y,X], axis=1)
data_spam.head()

,spam,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
0,1.0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,...,0.0,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61.0,278.0
1,1.0,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,...,0.0,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101.0,1028.0
2,1.0,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,...,0.0,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485.0,2259.0
3,1.0,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.0,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40.0,191.0
4,1.0,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,...,0.0,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40.0,191.0


In [23]:
# Divide the data into test and traing sets
random.seed(2015)
train_index=random.sample(list(range(len(X))),int(len(X)*0.8))
test_index=[x for x in list(range(len(X))) if x not in train_index]
X_train=X.iloc[train_index,:]
X_test=X.iloc[test_index,:]
y_train=y.iloc[train_index]
y_test=y.iloc[test_index]

### (1) Use the code provided to build your model on training data and report the os accuracy on your testing data.

a. what is the probability matrix (dp)?

b. what is the out of sample accuracy ?

### (2) Use Sklearn package to double check your solution. 

## Semi-supervised EM classifier

# Example 3. Taxi trip classification with partially missing labels

In [27]:
#read the same data for the taxi trip as used before
data = pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/ML_2017/NYC_taxi_sample.csv",index_col=0)
data.manhattan=2-data.manhattan
#data structure: categorical variables showing if the trip was within Mangattan and how big/small were the distance travelled, speed, tip left as well as how many passengers were on the trip
data.head()

,manhattan,tip,dist,speed,pass
26066,1,1,1,1,1
26100,1,3,5,6,1
26354,1,1,6,4,2
26589,0,2,2,2,1
26722,1,4,1,1,1


In [28]:
#as before take approximately 75% of the dataset for the training set, reserving the rest for the test set
np.random.seed(2014)
ind=stats.bernoulli.rvs(p = 0.75, size = len(data.index))
train=data[ind==1]
test=data[ind==0]
train.head()

,manhattan,tip,dist,speed,pass
26066,1,1,1,1,1
26100,1,3,5,6,1
26354,1,1,6,4,2
26722,1,4,1,1,1
26793,0,3,3,3,2


In [29]:
#split output variable from the features
X=train.iloc[:,1:]
y=train.iloc[:,0]
X_test=test.iloc[:,1:]
y_test=test.iloc[:,0]

In [30]:
#now let's randomly delete 99.5% of the labels from the training set
random.seed(2015)
Label_index=random.sample(list(range(len(X))),int(len(X)*0.005))
Unlabel_index=[x for x in list(range(len(X))) if x not in Label_index]

X_Label=X.iloc[Label_index,:]
X_Unlabel=X.iloc[Unlabel_index,:]   
y_Label=y.iloc[Label_index]
y_Unlabel=y.iloc[Unlabel_index]# we should not use this information

Let's see what happens if we rely only on the labeled data training Naive Bayes Classifier over it

In [31]:
trainData=pd.concat([y_Label,X_Label],axis=1)
len(y_Label)

19

In [32]:
dp=trainNaiveBayesDiscrete(trainData)
dp

[  manhattan        tip       dist      speed       pass
 1  0.736842        0.5   0.142857   0.142857   0.571429
 2       NaN  0.0714286   0.357143   0.428571   0.214286
 3       NaN   0.214286   0.214286  0.0714286   0.142857
 4       NaN  0.0714286   0.214286   0.285714  0.0714286
 5       NaN          0          0          0          0
 6       NaN   0.142857  0.0714286  0.0714286          0,
   manhattan  tip dist speed pass
 1  0.263158  0.6  0.2   0.2  0.8
 2       NaN    0    0   0.2    0
 3       NaN    0  0.2   0.4    0
 4       NaN  0.2  0.2     0  0.2
 5       NaN    0  0.2     0    0
 6       NaN  0.2  0.2   0.2    0]

In [33]:
testdata=pd.concat([y_test,X_test],axis=1)
C=classifyNaiveBayesDiscrete(testdata,dp)

In [34]:
acc=format(100.0*sum(C==y_test)/len(y_test))
print "We correctly classified {0} percents of the trips based on the labeled data only".format(acc)     

We correctly classified 50.8148148148 percents of the trips based on the labeled data only


In [67]:
#implementation of Expectation-Maximization algorithm for partially labeled data
import math
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)
    
 
    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))),
                        axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()
 
        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    if (((dp1[0]-dp[0])**2).sum()).sum()+(((dp1[1]-dp[1])**2).sum()).sum()<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels:    
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
            L=L+math.log(P)
        
        print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
    
  return dp



In [68]:
#perform EM estimation for theta
dpEM=EM(X_Label,y_Label,X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(testdata,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y_test)/len(y_test)
print "After EM we correctly classified {0} percents of the trips".format(acc)

C:\Users\lingjing\Anaconda2\lib\site-packages\ipykernel\__main__.py:32: RuntimeWarning: invalid value encountered in divide
C:\Users\lingjing\Anaconda2\lib\site-packages\ipykernel\__main__.py:33: RuntimeWarning: invalid value encountered in divide


Iteration 1: log maximum liklihood = -23104.910287
Iteration 2: log maximum liklihood = -22956.539844
Iteration 3: log maximum liklihood = -22733.3814326
Iteration 4: log maximum liklihood = -22418.4482997
Iteration 5: log maximum liklihood = -22144.6817837
Iteration 6: log maximum liklihood = -22030.6757311
Iteration 7: log maximum liklihood = -21997.1619209
Iteration 8: log maximum liklihood = -21981.4247846
After EM we correctly classified 67.2592592593 percents of the trips


# Unsupervised EM clustering


# Example 4. Taxi trip clustering with no labels


In [69]:
X_Unlabel=X #take all observations unlabeled

In [70]:
np.random.seed(2016)
#initialize theta randomly
dp[0].iloc[0,0]=np.random.uniform(0,1)
dp[1].iloc[0,0]=dp[1].iloc[0,0]=1-dp[0].iloc[0,0]
for j in range(1,len(dp[0].T)):
    b=np.random.uniform(0,1,len(dp[0]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[0].iloc[:,j]=b
for j in range(1,len(dp[1].T)):
    b=np.random.uniform(0,1,len(dp[1]))
    b=np.asarray(b)/float(np.asarray(b).sum())
    dp[1].iloc[:,j]=b


In [71]:
dp

[  manhattan       tip      dist     speed      pass
 1  0.896705  0.203711  0.233433  0.052900  0.275895
 2       NaN  0.218506  0.246143  0.221832  0.275194
 3       NaN  0.206894  0.205944  0.216376  0.020212
 4       NaN  0.128907  0.191068  0.025485  0.103416
 5       NaN  0.179253  0.079939  0.346481  0.109967
 6       NaN  0.062729  0.043473  0.136927  0.215316,
   manhattan       tip      dist     speed      pass
 1  0.103295  0.145768  0.248003  0.008330  0.138931
 2       NaN  0.118057  0.187649  0.327176  0.128262
 3       NaN  0.159788  0.189117  0.167362  0.298924
 4       NaN  0.346520  0.101663  0.130922  0.113730
 5       NaN  0.161480  0.147634  0.289818  0.229165
 6       NaN  0.068387  0.125933  0.076393  0.090988]

In [72]:
#perform EM estimation for theta
dpEM=EM([],[],X_Unlabel,dp)
#OS test
C=classifyNaiveBayesDiscrete(testdata,dpEM) #classify test data with a new theta given by EM
acc=100.0*sum(C==y_test)/len(y_test)
if acc<50: #if most of the labels are misclassified it is just the matter of swapping the labels 1<->0
    acc=100-acc
print "After EM we correctly classified {0} percents of the trips".format(acc)

Iteration 1: log maximum liklihood = -23151.4496838
Iteration 2: log maximum liklihood = -23102.1686162
Iteration 3: log maximum liklihood = -23066.4449969
Iteration 4: log maximum liklihood = -23038.4978789
Iteration 5: log maximum liklihood = -23007.8977578
Iteration 6: log maximum liklihood = -22947.8175748
Iteration 7: log maximum liklihood = -22799.4324681
Iteration 8: log maximum liklihood = -22511.6032021
Iteration 9: log maximum liklihood = -22203.7134995
Iteration 10: log maximum liklihood = -22039.6582528
Iteration 11: log maximum liklihood = -21981.8045475
Iteration 12: log maximum liklihood = -21957.5491092
Iteration 13: log maximum liklihood = -21943.1487191
After EM we correctly classified 67.7037037037 percents of the trips
